In [1]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd4843 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 24 × 12th Gen Intel(R) Core(TM) i9-12900K
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, alderlake)
Threads: 4 default, 0 interactive, 2 GC (on 24 virtual cores)
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
using Pkg; Pkg.activate("."); #Pkg.instantiate(); Pkg.precompile(); 
Pkg.status()

  Activating project at `C:\Users\o6m1g\Documents\GitHub\SMC-Election-private`


Status `C:\Users\o6m1g\Documents\GitHub\SMC-Election-private\Project.toml`
⌃ [0bf59076] AdvancedHMC v0.8.1
⌃ [c88b6f0a] BridgeStan v2.6.2
  [336ed68f] CSV v0.10.15
⌃ [a93c6f00] DataFrames v1.7.0
  [31c24e10] Distributions v0.25.120
  [38e38edf] GLM v1.9.0
  [4138dd39] JLD v0.13.5
  [682c06a0] JSON v0.21.4
  [b964fa9f] LaTeXStrings v1.4.0
  [6fdf6af0] LogDensityProblems v2.1.2
  [86f7a689] NamedArrays v0.10.4
  [ce719bf2] PSIS v0.9.8
⌃ [91a5bcdd] Plots v1.40.18
⌃ [92933f4c] ProgressMeter v1.10.4
  [df47a6cb] RData v1.1.0
  [2913bbd2] StatsBase v0.34.6
  [4c63d2b9] StatsFuns v1.5.0
  [f3b207a7] StatsPlots v0.15.7
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [2f01184e] SparseArrays v1.10.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
N_THREADS = Threads.nthreads()
@assert N_THREADS == 4 N_THREADS

In [4]:
include("Init.jl")

In [5]:
draws_0 = load(joinpath("input", "draws_$(RUN).jld"))["data"]
draws_0 = NamedArray(
    draws_0,
    (axes(draws_0,1), param_unc_names(model_0)),
    (:r, :d),
)
stats_0 = CSV.read(joinpath("input", "stats-df_$(RUN).csv"), DataFrame);

# SMC

In [6]:
function SMCS_Stan(draws_0::typeof(draws_0), ℓπ_vec::Vector{LogDensity})
    
    # Define problem dimensions
    R = N = size(draws_0, 1)
    L = length(ℓπ_vec) - 1
    
    names_vec = [param_unc_names(ℓπ.model) for ℓπ in ℓπ_vec]
    D_vec = LogDensityProblems.dimension.(ℓπ_vec)

    # Initialize containers
    ℓπ_vec      = NamedArray(ℓπ_vec,          0:L,                :l       )
    names_vec   = NamedArray(names_vec,       0:L,                :l       )
    D_vec       = NamedArray(D_vec,           0:L,                :l       )
    
    particles   = NamedArray([
            NamedArray(zeros(N, D_vec[:l => ℓ]),
                        (1:N, names_vec[:l => ℓ]),
                        (:n,:d)
            ) for ℓ in 0:L], 0:L, :l)
    log_weights = NamedArray(zeros(1+L,N),   (0:L, 1:N),         (:l,:n)   )
    weights     = NamedArray(zeros(1+L,N),   (0:L, 1:N),         (:l,:n)   )
    
    k̂           = NamedArray(zeros(1+L),      0:L,                :l       )
    ESS         = NamedArray(zeros(1+L),      0:L,                :l       )
    mcmc_flag   = NamedArray(zeros(1+L),      0:L,                :l       )
    times       = NamedArray(zeros(1+L),      0:L,                :l       )
    
    for n in 1:N
        particles[:l => 0][:n => n, :d => names_vec[:l => 0]] = draws_0[n, names_vec[:l => 0]] |> Vector{Float64}
        #particles[:l => 0][:n => n, :d => names_vec[:l => 0]] = param_unconstrain(
        #    ℓπ_vec[:l => l].model, draws_0[n, names_vec[:l => 0]] |> Vector{Float64})
    end

    # Log ratio
    function log_G(l::Int, n::Int)::Float64
        log_γ_0 = log_density(ℓπ_vec[:l => l-1].model,
            particles[:l => l][:n => n, :d => names_vec[:l => l-1]])
        log_γ_1 = log_density(ℓπ_vec[:l => l].model,
            particles[:l => l][:n => n, :d => names_vec[:l => l]])
        log_γ_1 - log_γ_0
    end
    
    # Log weights normalizer
    function _normalize(_log_w::Vector{Float64})::Vector{Float64}
        _w = exp.(_log_w .- maximum(_log_w))
        _w = _w / sum(_w)
        _w
    end
    
    # ESS computer
    function _ess(_w::Vector{Float64})::Float64
        1 / sum(@. exp(2 * log(_w)))
    end
    
    # MCMC kernel
    initial_ϵ, n_steps = stats_0.step_size[end], stats_0.n_steps[end]
    function _move(initial_θ::Vector{Float64},
            n_samples::Int, n_adapts::Int,
            ℓπ::LogDensity,
            mass::Vector{Float64})::Vector{Float64}
        metric = DiagEuclideanMetric(mass)
        hamiltonian = Hamiltonian(metric, ℓπ)
        integrator = Leapfrog(initial_ϵ)
        adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))
        kernel = HMCKernel(Trajectory{EndPointTS}(integrator, FixedNSteps(n_steps)))
        rng = Random.TaskLocalRNG()
        samples, stats = sample(rng, hamiltonian, kernel, initial_θ, n_samples, adaptor, n_adapts;
            verbose=false, progress=false)
        samples[end]
    end

    function _record(l::Int; _log_weights::Vector{Float64})
        log_weights[:l => l] = _log_weights
        weights[:l => l] = log_weights[:l => l] |> _normalize
        ESS[:l => l] = weights[:l => l] |> _ess
        k̂[:l => l] = psis(log_weights[:l => l]).pareto_shape
    end

    # Sequential Monte Carlo
    for l in 0:L
        times[:l => l] = @elapsed begin   
            if l == 0 # Obtain prior draw
                mcmc_flag[:l => l] = true
                _record(l; _log_weights=repeat([-log(N)], N))
            
            elseif l ≥ 1
                # Copy "same-name" columns
                particles[:l => l][:d => names_vec[:l => l-1]] = particles[:l => l-1][:d => names_vec[:l => l-1]]
                # Propose "new-name" columns
                names_diff = setdiff(names_vec[:l => l], names_vec[:l => l-1])
                if length(names_diff) > 0
                    @info names_diff
                    # Assumes prior is "raw_..." ~ Normal(): true for our model
                    particles[:l => l][:d => names_diff] = rand(Normal(), N, length(names_diff))
                end
                # Evaluate proposed
                _log_weights = log_weights[:l => l-1] + [log_G(l, n) for n in 1:N]
                _record(l; _log_weights=_log_weights)
                
                if ESS[:l => l] >= N/2 # Delta kernel
                    @info "Delta Kernel ($(l)/$(L); ESS=$(ESS[:l => l] |> round |> Int)/$(N))"
                    mcmc_flag[:l => l] = false
                    # particles[:l => l] = particles[:l => l-1] done above
                    # _log_weights = as is
                
                else # MCMC kernel
                    @info "MCMC Kernel ($(l)/$(L); ESS=$(ESS[:l => l] |> round |> Int)/$(N))"
                    mcmc_flag[:l => l] = true
                    
                    # MCMC move
                    A_0 = wsample(1:N, _log_weights |> _normalize, N)
                    _begin = particles[:l => l][:n => A_0, :d => names_vec[:l => l]] # expanded (l-1) copied to (l) above
                    _end = _begin |> similar
                    mass = _var(particles[:l => l][:n => 1:N, :d => names_vec[:l => l]]; dims=1)
                    mass[mass .≤ 0] .= median(mass[mass .> 0])
                    
                    N_cut = N; p = ProgressMeter.Progress(N_cut); @Threads.threads for n in 1:N_cut
                        _end[n,:] = _move(_begin[n,:], 3, 0, ℓπ_vec[:l => l], mass)
                        ProgressMeter.next!(p)
                    end
                    particles[:l => l][:n => 1:N, :d => names_vec[:l => l]] = _end
                    
                    _log_weights = repeat([-log(N)], N)
                end
                _record(l; _log_weights=_log_weights)
            end
        end
    end
    
    (; ℓπ_vec, names_vec, D_vec, 
        particles, weights,
        ESS, k̂, mcmc_flag,
        R, N, L,
        times
    )
end

SMCS_Stan (generic function with 1 method)

## Fundamentals-based forecast

In [ ]:
Random.seed!(1)
#results_1_pos = SMCS_Stan(draws_0, perturb_1(s="CA")[1:2] |> return_LogDensities); # pilot run
results_1_pos = SMCS_Stan(draws_0, perturb_1(s="CA") |> return_LogDensities);

In [ ]:
Random.seed!(1)
results_1_neg = SMCS_Stan(draws_0, perturb_1(s="CA", negative=true) |> return_LogDensities);

In [ ]:
save_SMC(results_1_pos, "1")
save_SMC(results_1_neg, "1-neg")

## Random-walk scale

In [ ]:
Random.seed!(1)
#results_2_pos = SMCS_Stan(draws_0, perturb_2()[1:2] |> return_LogDensities); # pilot run
results_2_pos = SMCS_Stan(draws_0, perturb_2() |> return_LogDensities);

In [ ]:
Random.seed!(1)
results_2_neg = SMCS_Stan(draws_0, perturb_2(negative=true) |> return_LogDensities);

In [ ]:
save_SMC(results_2_pos, "2")
save_SMC(results_2_neg, "2-neg")

## Fundamentals-based forecast informativity

In [ ]:
Random.seed!(1)
#results_3 = SMCS_Stan(draws_0, perturb_3()[1:2] |> return_LogDensities); # pilot run
results_3 = SMCS_Stan(draws_0, perturb_3() |> return_LogDensities);

In [ ]:
Random.seed!(1)
#results_3_inv = SMCS_Stan(draws_0, perturb_3(inverse=true)[1:2] |> return_LogDensities); # pilot run
results_3_inv = SMCS_Stan(draws_0, perturb_3(inverse=true) |> return_LogDensities);

In [ ]:
save_SMC(results_3,     "3")
save_SMC(results_3_inv, "3-inv")

## Data-insertion example

In [ ]:
Random.seed!(1)
results_4 = SMCS_Stan(
    draws_0,
    perturb_4(30;
        state="PA", n_dem=300, n=600, date=RUN + Day(10),
        poll_mode=1, poll_pop_state=3, pollster="NBC"
    ) |> return_LogDensities
);

In [ ]:
save_SMC(results_4,     "4")

# Figures

In [ ]:
# save_SMC(results_1_pos, "1")
# save_SMC(results_1_neg, "1-neg")
# save_SMC(results_2_pos, "2")
# save_SMC(results_2_neg, "2-neg")
# save_SMC(results_3,     "3")
# save_SMC(results_3_inv, "3-inv")
# save_SMC(results_4,     "4")

In [ ]:
results_1_pos = load_SMC("1")
results_1_neg = load_SMC("1-neg")
results_2_pos = load_SMC("2")
results_2_neg = load_SMC("2-neg")
results_3     = load_SMC("3")
results_3_inv = load_SMC("3-inv")
results_4     = load_SMC("4")
;

# Accuracy

In [ ]:
let
    function _plot(perturb_i::String, results)
        
        (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results

        l = L
        DIR = joinpath("output", "$(perturb_i)", "mcmc", "draws_$(RUN)_ℓ-$(l).jld")
        mcmc = load(DIR)["data"]
        smc = particles[:l => l] |> Matrix

        #base_est = _mean(particles[:l => 0] |> Matrix; dims=1)
        smc_est = _sum(smc .* weights[:l => l]; dims=1)
        mcmc_est = _mean(mcmc; dims=1)
    
        (lim_min, lim_max) = extrema(abs.([mcmc_est; smc_est]))
        plot([-lim_max, lim_max], [-lim_max, lim_max], color=:black)
        #scatter!(base_est, mcmc_est, legend=false, color=:black, alpha=0.8,
        #    ms=2.5, mswidth=1)
        scatter!(smc_est, mcmc_est, legend=false, color=:white, alpha=0.8,
            # markershape=:+, alpha=(abs.(mcmc_est) ./ lim_max),
            ms=1.5, mswidth=1)
        LinReg_OLS = lm(smc_est[:,:], mcmc_est)
        β_hat = coef(LinReg_OLS)[1]
        #plot!(-lim_max:lim_max, x -> β_hat * x, color=:blue)
        #annotate!(lim_max*2/3, -lim_max*2/3, text(L"R^2=" * "$(round(adjr2(LinReg_OLS)[1]; digits=2))", 8))
        #annotate!(1, 1, text("$(cor(smc_est, mcmc_est))"))
        (xlabel, ylabel) = perturb_i == "4" ? ("SMC", "Bruteforce") : (" ", "")
        plot!(xlabel=xlabel, xguidefontcolor=:red, xtickfontcolor=:red,
            ylabel=ylabel, yguidefontcolor=:deepskyblue, ytickfontcolor=:deepskyblue)
    end
    
    plot(_plot("4", results_4), _plot("1", results_1_pos), _plot("3", results_3), _plot("2", results_2_pos),
        plot(axis=false, grid=false), _plot("1-neg", results_1_neg), _plot("3-inv", results_3_inv), _plot("2-neg", results_2_neg),
        layout=grid(2,4), size=(170*4,200*2),
        leftmargin=Plots.mm .* [repeat([2], 2*1); repeat([-1], 2*3)] |> permutedims,
        plot_title="Last step posterior means (unconstrained)",
        # title=["\n                   #1" "\n                   #2" "\n                   #3" "\n                   #4" "\n                   #2" "\n                   #3" "\n                   #4"],
        title=["\nData insertion" "\nPrior forecast (+)" "\nConfidence (+)" "\nWalk scale (+)" " " "\nPrior forecast (-)" "\nConfidence (-)" "\nWalk scale (-)"],
        titlefontsize=10,
    ) |> display

    DIR = joinpath("img"); isdir(DIR) || mkpath(DIR)
    savefig(joinpath(DIR, "accuracy.pdf"))
end;

In [ ]:
let
    function _sim_runtime(time_vec::Vector{<:Real}, num_machines::Int)
        n_tasks = length(time_vec)
        machine_available = fill(0.0, num_machines)  # current time per machine
        sim_time_vec = similar(time_vec)  # output runtimes
        for i in 1:n_tasks
            #@info i, time_vec[i]
            #@info machine_available
            # Find the machine that becomes available the earliest
            m = argmin(machine_available)
            start_time = machine_available[m]
            sim_time_vec[i] = start_time + time_vec[i]
            machine_available[m] = sim_time_vec[i]  # update machine's availability
            #@info machine_available
        end
        sim_time_vec
    end
    
    function _plot(perturb_pos, perturb_neg, results_pos, results_neg)
    
        L = max(results_pos[:L], results_neg[:L])
        
        times_smc_pos  = results_pos[:times] / 60
        times_smc_neg  = results_neg[:times] / 60
        times_mcmc_pos = load(joinpath("output", "$(perturb_pos)", "mcmc", "runtime_$(RUN).jld"))["data"] / 60
        times_mcmc_neg = load(joinpath("output", "$(perturb_neg)", "mcmc", "runtime_$(RUN).jld"))["data"] / 60
        
        (lim_min, lim_max) = extrema([times_mcmc_pos; times_smc_pos; times_mcmc_neg; times_smc_neg])
        lim_max *= 1.1
        plot([0, lim_max], [0, lim_max], color=:black, linestyle=:dot)
        plot!(LinRange(0, lim_max, 2), LinRange(0, lim_max, 2), fillrange=repeat([lim_max], 2),
            color=:deepskyblue, alpha=0.15)
        plot!(LinRange(0, lim_max, 2), LinRange(0, lim_max, 2), fillrange=repeat([0], 2),
            color=:red, alpha=0.15)
        scatter!(times_mcmc_neg, times_smc_neg[:l => 1:results_neg[:L]], alpha=1.0, color=:white)
        scatter!(times_mcmc_pos, times_smc_pos[:l => 1:results_pos[:L]], alpha=1.0, color=:white)
        plt_1 = plot!(legend=false,
            xlabel=perturb_pos == "3" ? repeat(" ", 30) * "Bruteforce run time" : " ", xguidefontcolor=:deepskyblue, xtickfontcolor=:deepskyblue,
            ylabel=" ", yguidefontcolor=:red, ytickfontcolor=:red,
            xlim=(0, lim_max), ylim=(0, lim_max))
    
        time_smc_pos = cumsum(times_smc_pos; dims=1)
        time_smc_neg = cumsum(times_smc_neg; dims=1)
        plot(0:results_pos[:L], time_smc_pos, markershape=:o, ms=1.5, mswidth=0, color=:red)
        plot!(0:results_neg[:L], time_smc_neg, linestyle=:dot, markershape=:o, ms=1.5, mswidth=0, color=:red)
        #annotate!(L, max(time_smc_pos[end], time_smc_neg[end]), text("Approx.   \n", 8, :red, :right))
        time_mcmc = _sim_runtime([times_mcmc_neg[end:-1:begin]; times_mcmc_pos], N_THREADS)[end]
        hline!([time_mcmc], color=:deepskyblue)

        annotate!(1, time_mcmc, text("\n4 ma", 7, :deepskyblue, :left))
        
        min_threads, time_mcmc_pos = 0, Inf
        while time_smc_pos[end] < time_mcmc_pos
            min_threads += 1
            time_mcmc_pos = _sim_runtime(times_mcmc_pos, min_threads)[end]
            min_threads ≥ L && break
        end

        min_threads, time_mcmc_neg = 0, Inf
        while time_smc_neg[end] < time_mcmc_neg
            min_threads += 1
            time_mcmc_neg = _sim_runtime(times_mcmc_neg, min_threads)[end]
            min_threads ≥ L && break
        end

        time_mcmc = max(time_mcmc_pos, time_mcmc_neg)
        hline!([time_mcmc], color=:deepskyblue)
        annotate!(1, time_mcmc, text("$(min_threads) ma\n", 7, :deepskyblue, :left))
        
        plt_2 = plot!(legend=false, xlim=(0, xlims(plot!())[end]), ylim=(0, ylims(plot!())[end]),
            xticks=0:5:L, xlabel=perturb_pos == "3" ? repeat(" ", 30) * "Meshes" : " ", ylabel="")
        
        plot(plt_1, plt_2, layout=grid(2,1))
    end

    function _plot(perturb_i, results)
        
        L = results[:L]
        
        times_smc  = results[:times] / 60
        times_mcmc = load(joinpath("output", "$(perturb_i)", "mcmc", "runtime_$(RUN).jld"))["data"] / 60
        
        (lim_min, lim_max) = extrema([times_mcmc; times_smc])
        lim_max *= 1.1
        plot([0, lim_max], [0, lim_max], color=:black, linestyle=:dot)
        plot!(LinRange(0, lim_max, 2), LinRange(0, lim_max, 2), fillrange=repeat([lim_max], 2),
            color=:deepskyblue, alpha=0.15)
        plot!(LinRange(0, lim_max, 2), LinRange(0, lim_max, 2), fillrange=repeat([0], 2),
            color=:red, alpha=0.15)
        annotate!(lim_max*4.8/5, lim_max*3.2/5, text("SMC\nfaster", 7, :red, :right))
        annotate!(lim_max*3/5, lim_max*4.2/5, text("Bruteforce\nfaster", 7, :deepskyblue, :right))
        scatter!(times_mcmc, times_smc[:l => 1:results[:L]], alpha=0.7, color=:white)
        plt_1 = plot!(legend=false,
            xlabel=" ", xguidefontcolor=:deepskyblue, xtickfontcolor=:deepskyblue,
            ylabel="SMC run time", yguidefontcolor=:red, ytickfontcolor=:red,
            xlim=(0, lim_max), ylim=(0, lim_max))
    
        time_smc = cumsum(times_smc; dims=1)
        plot(0:results[:L], time_smc, markershape=:o, ms=1.5, mswidth=0, color=:red)
        annotate!(L, time_smc[end], text("\n SMC\n (4 ma)", 7, :red, :left))
        
        time_mcmc = _sim_runtime(times_mcmc, N_THREADS)[end]
        hline!([time_mcmc], color=:deepskyblue)
        annotate!(1, time_mcmc, text("\n4 machines", 7, :deepskyblue, :left))

        min_threads, time_mcmc = 0, Inf
        while time_smc[end] < time_mcmc
            min_threads += 1
            time_mcmc = _sim_runtime(times_mcmc, min_threads)[end]
            min_threads ≥ L && break
        end
        hline!([time_mcmc], color=:deepskyblue)
        annotate!(1, time_mcmc, text("$(min_threads) ma\n", 7, :deepskyblue, :left))
        
        plt_2 = plot!(legend=false, 
            xlim=(0, xlims(plot!())[end]), ylim=(0, ylims(plot!())[end]),
            xticks=0:5:L, xlabel=" ", ylabel="Cumulative")
        
        plot(plt_1, plt_2, layout=grid(2,1))
    end
    
    #_plot("3", "3-inv", results_3, results_3_inv), size=(230,200*2), plot_title="Run time #1"
    plot(
        _plot("4", results_4),
        _plot("3", "3-inv", results_3, results_3_inv),
        _plot("1", "1-neg", results_1_pos, results_1_neg),
        _plot("2", "2-neg", results_2_pos, results_2_neg),
        layout=grid(1,4), tick_direction=:out, size=(170*4,180*2),
        plot_title="Run times [min.]",
        title=["\nData insertion" "" "\nPrior forecast" "" "\nConfidence" "" "\nWalk scale" ""],
        titlefontsize=10,
        bottommargin=[6Plots.mm 2Plots.mm],
        leftmargin=[repeat([2Plots.mm], 2*1); repeat([0Plots.mm], 2*3)] |> permutedims,
        #dpi=300,
    ) |> display

    DIR = joinpath("img"); isdir(DIR) || mkpath(DIR)
    savefig(joinpath(DIR, "runtimes.pdf"))
end;

In [ ]:
function plot_data(s::String)
    
    plot()
    
    df_s = filter(row -> row.state == s, df)
    bottom_flag = 0
    for (i, (x_i, y_i, n_i)) in enumerate(zip((START:END)[df_s.index_t], df_s.pct_clinton, df_s.n_respondents))
        a_i = (.4 + (1 - 0.4)*(n_i/maximum(df_s.n_respondents)))
        scatter!([x_i], [y_i], mswidth=0, color=:black, ms=4a_i, alpha=a_i)
        if n_i ≥ quantile(df_s.n_respondents, 0.9)
            #annotate!(x_i, y_i, text("$(n_i)", 6, bottom_flag % 2 != 0 ? :bottom : :top))
            #bottom_flag += 1
        end
    end

    hline!([0.5], color=:black, alpha=0.2)
    
    ymin, ymax = ylims(plot!())
    #annotate!(RUN, ymin, text(" $(END - RUN) out", :left, 7))
    
    vline!([RUN, END], color=:black)
    vspan!([RUN, END], color=:gray, alpha=0.1)
    
    xticks = END:Day(-7*8):START |> unique |> sort
    plot!(title=abbrev2full[s],
        tickdirection=:out, #frame=:semi,
        legend=false,
        xrot=30, xticks=(xticks, string.(xticks)), #xtickfontsize=5,
        #ylabel=L"\leftarrow" * "Rep." * repeat(" ", 9) * "Dem." * L"\rightarrow",
        #ylim=(max(0, ymin - 0.05), min(1, ymax + 0.05)),
        bottommargin=3Plots.mm, rightmargin=8Plots.mm
    )
end

In [ ]:
# perturb_i = 1

Random.seed!(1); let
    
    function _plot(s::String)
        
        plot_data(s)
        
        for (i, results) in [results_1_pos, results_1_neg] |> enumerate
            
            (; ℓπ_vec, names_vec, D_vec,
                particles, weights,
                ESS, k̂, mcmc_flag,
                R, N, L,
                times) = results
            N_cut = N
            
            @showprogress for l in LinRange(0, L, 4) .|> round .|> Int

                c_l = l == 0 ? :black : (i == 1 ? :blue : :red)
                
                function _plot_trend(θ::Matrix{Float64}, w::Vector{Float64})
                    _weighted_quantile(θ_k::Vector{Float64}, w::Vector{Float64}; α::Float64) = (
                        sorted_idx = sortperm(θ_k);
                        θ_k[sorted_idx][findall(cumsum(w[sorted_idx]) .< α)[end]])
                
                    l == L && plot!(START:END, [_weighted_quantile(θ[:,k], w; α=0.1/2) for k in axes(θ,2)],
                        fillrange=[_weighted_quantile(θ[:,k], w; α=1-0.1/2) for k in axes(θ,2)],
                        color=c_l, alpha=0.15, linewidth=0)
                    plot!(START:END, _sum(θ .* w; dims=1), color=c_l, alpha=l == 0 ? 1 : l/L)
                end
                
                particles_l = NamedArray(
                    [param_constrain(ℓπ_vec[:l => l].model, particles[:l => l][:n => n];
                            include_tp=true,
                            include_gq=true, rng=StanRNG(ℓπ_vec[:l => l].model, SEED)
                            ) for n in 1:N_cut] |> vecvec2mat,
                    (1:N_cut,
                        param_names(ℓπ_vec[:l => l].model;
                            include_tp=true, include_gq=true)),
                    (:n, :d))
                
                θ = zeros(N_cut, data_0["T"])
                for (n,t) in Iterators.product(1:N_cut, 1:data_0["T"])
                    θ[n,t] = particles_l[:n => n, :d => "predicted_score.$(t).$(abbrev2int[s])"]
                end
                _plot_trend(θ, weights[:l => l])
                
                m_s = ℓπ_vec[:l => l].data["mu_b_prior"][abbrev2int[s]] |> logistic
                if s == "CA"
                    if l == 0
                        annotate!(END + Day(35), m_s,
                            i == 1 ? text(L"\uparrow", :bottom, :blue) : text(L"\downarrow", :top, :red))
                    end
                    scatter!([END + Day(15)], [m_s],
                            markershape=:hline, color=c_l, alpha=l == 0 ? 1 : l/L)
                else
                    l == 0 && scatter!([END + Day(15)], [m_s],
                            markershape=:hline, color=c_l, alpha=1)
                end
            end
        end
        plot!()
    end
    
    plot(_plot("CA"),
        (_plot("AR"); plot!(xlabel="")),
        (_plot("OH"); plot!(xlabel="")),
        layout=grid(1,3))
    #for s in keys(abbrev2int)
    #    @info s; _plot(s) |> display; a
    #end
    #_plot("CA")
end;

In [ ]:
plt_1 = plot!(size=(700,250),
    leftmargin=[5 -3 -7] .* Plots.mm,
    bottommargin=6Plots.mm,
    dpi=300
)

In [ ]:
savefig(joinpath("img", "fundamental-location.pdf"));

In [ ]:
# perturb_i = 3
Random.seed!(1); let
    
    function _plot(s::String)
        
        plot_data(s)
        
        for (i, results) in [results_3, results_3_inv] |> enumerate
            
            (; ℓπ_vec, names_vec, D_vec,
                particles, weights,
                ESS, k̂, mcmc_flag,
                R, N, L,
                times) = results
            N_cut = N
            
            @showprogress for l in LinRange(0, L, 2) .|> round .|> Int

                c_l = l == 0 ? :black : (i == 1 ? :darkred : :green)
                
                function _plot_trend(θ::Matrix{Float64}, w::Vector{Float64})
                    _weighted_quantile(θ_k::Vector{Float64}, w::Vector{Float64}; α::Float64) = (
                        sorted_idx = sortperm(θ_k);
                        θ_k[sorted_idx][findall(cumsum(w[sorted_idx]) .< α)[end]])
                
                    l == L && plot!(START:END, [_weighted_quantile(θ[:,k], w; α=0.1/2) for k in axes(θ,2)],
                        fillrange=[_weighted_quantile(θ[:,k], w; α=1-0.1/2) for k in axes(θ,2)],
                        color=c_l, alpha=0.15, linewidth=0)
                    plot!(START:END, _sum(θ .* w; dims=1), color=c_l, alpha=l == 0 ? 1 : l/L)
                end
                
                particles_l = NamedArray(
                    [param_constrain(ℓπ_vec[:l => l].model, particles[:l => l][:n => n];
                            include_tp=true,
                            include_gq=true, rng=StanRNG(ℓπ_vec[:l => l].model, SEED)
                            ) for n in 1:N_cut] |> vecvec2mat,
                    (1:N_cut,
                        param_names(ℓπ_vec[:l => l].model;
                            include_tp=true, include_gq=true)),
                    (:n, :d))
                
                θ = zeros(N_cut, data_0["T"])
                for (n,t) in Iterators.product(1:N_cut, 1:data_0["T"])
                    θ[n,t] = particles_l[:n => n, :d => "predicted_score.$(t).$(abbrev2int[s])"]
                end
                _plot_trend(θ, weights[:l => l])

                m_s = ℓπ_vec[:l => l].data["mu_b_prior"][abbrev2int[s]] |> logistic
                s_s = ℓπ_vec[:l => l].data["mu_b_T_scale"] / 2
                δ = Day(Int(round((i == 1 ? +1 : -1) * 10 * l/L)))
                l == 0 && scatter!([END + Day(20)], [m_s],
                            markershape=:hline, color=c_l, alpha=l == 0 ? 1 : l/L)
                plot!([END + Day(20) + δ, END + Day(20) + δ], [m_s - s_s, m_s + s_s], color=c_l)
            end
        end
        plot!()
    end
    
    plot(_plot("CA"),
        (_plot("AR"); plot!(xlabel="")),
        (_plot("OH"); plot!(xlabel="")),
        layout=grid(1,3))
    #for s in keys(abbrev2int)
    #    @info s; _plot(s) |> display#; a
    #end
    #_plot("CA")
end;

In [ ]:
plt_2 = plot!(size=(700,250),
    leftmargin=[5 -3 -7] .* Plots.mm,
    bottommargin=6Plots.mm,
)

In [ ]:
savefig(joinpath("img", "fundamental-scale.pdf"));

In [ ]:
# perturb_i = 3
let    
    _weighted_quantile(θ_k::Vector{Float64}, w::Vector{Float64}; α::Float64) = (
        _sorted_idx = sortperm(θ_k);
        θ_k[_sorted_idx][findall(cumsum(w[_sorted_idx]) .< α)[end]])

    function _return_posteriors(; l::Int)
        particles_l = NamedArray(
            [param_constrain(ℓπ_vec[:l => l].model, particles[:l => l][:n => n];
                    include_tp=true,
                    include_gq=true, rng=StanRNG(ℓπ_vec[:l => l].model, SEED)
                    ) for n in 1:N] |> vecvec2mat,
            (1:N,
                param_names(ℓπ_vec[:l => l].model;
                    include_tp=true, include_gq=true)),
            (:n, :d))
        L_vec, M_vec, U_vec = Float64[], Float64[], Float64[]
        for s in 0:data_0["S"]
            if s == 0
                θ_s = Vector{Float64}([particles_l[:n => n, :d => "predicted_score.$(data_0["T"]).$(s)"]
                        for n=1:N, s=1:data_0["S"]] * data_0["state_weights"])
            else
                θ_s = particles_l[:d => "predicted_score.$(data_0["T"]).$(s)"]
            end
            _L = _weighted_quantile(θ_s, weights[:l => l]; α=0.1)
            _M = θ_s' * weights[:l => l]
            _U = _weighted_quantile(θ_s, weights[:l => l]; α=1-0.1)
            push!(L_vec, _L); push!(M_vec, _M); push!(U_vec, _U)
        end
        (L_vec, M_vec, U_vec)
    end
    
    (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results_3
    global plot_mat = NamedArray(zeros(1+data_0["S"], 1+2L, 3),
        (0:data_0["S"], -L:L, [:L, :M, :U]), (:s, :l, :type))
    @showprogress for l in 0:L#LinRange(0, L, L) .|> round .|> Int
        (L_vec, M_vec, U_vec) = _return_posteriors(l=l)
        plot_mat[:l => l, :type => :L] = L_vec
        plot_mat[:l => l, :type => :M] = M_vec
        plot_mat[:l => l, :type => :U] = U_vec
    end

    (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results_3_inv
    @showprogress for l in 1:L#LinRange(0, L, L)[2:end] .|> round .|> Int
        (L_vec, M_vec, U_vec) = _return_posteriors(l=l)
        plot_mat[:l => -l, :type => :L] = L_vec
        plot_mat[:l => -l, :type => :M] = M_vec
        plot_mat[:l => -l, :type => :U] = U_vec
    end

    global plot_mat_1 = plot_mat
end

In [ ]:
let
    plot_mat = plot_mat_1
    h = 0.42
    #sorted_idx_0 = [abbrev2int[s] for s in ["UT", "AL", "TX", "AK", "GA", "CO", "MD"]]
    sorted_idx_0 = sortperm(plot_mat[:l => 0, :type => :M]) .- 1
    
    hline([0.5], color=:black, alpha=0.2, label="")
    for (i, s_i) in sorted_idx_0 |> enumerate
        plot!(LinRange(i-h, i+h, size(plot_mat, 2)),
            plot_mat[:s => s_i, :type => :L],
            label=i == 1 ? "80% CI, lined-up horizontally (" * L"\leftarrow" * "tighter; looser" * L"\rightarrow" * ")" : "",
            fillrange=plot_mat[:s => s_i, :type => :U],
            color=s_i == 0 ? :black : :gray, alpha=0.5, linewidth=0)
        scatter!([i], [plot_mat[:s => s_i, :l => 0, :type => :M]],
            label=i == 1 ? "Original final score" : "",
            color=:black, ms=3, mswidth=1)

        options = (; color=:black, markershape=:hline, ms=4, mswidth=2, label=i == 1 ? "Prior forecast" : "")
        # R code: national_mu_prior <- weighted.mean(arm::invlogit(mu_b_prior), state_weights)
        m_s = s_i == 0 ? logistic.([results_2_pos[:ℓπ_vec][:l => 0].data["mu_b_prior"][s_for] for s_for in 1:data_0["S"]])' * data_0["state_weights"] : logistic(results_2_pos[:ℓπ_vec][:l => 0].data["mu_b_prior"][s_i])
        scatter!([i], [m_s]; options...)
    end
    S_vec = [s_i == 0 ? "National" : int2full[s_i] for s_i in sorted_idx_0]
    global plt_3 = plot!(size=(670,400), frame=:semi,
        xrot=-90, xticks=(1:length(S_vec), S_vec),
        bottommargin=10Plots.mm,
        legend=true,
        title="Adjusting final scale"
    )
end

In [ ]:
savefig(joinpath("img", "fundamental-scale-2.pdf"));

In [ ]:
# perturb_i = 2
let    
    _weighted_quantile(θ_k::Vector{Float64}, w::Vector{Float64}; α::Float64) = (
        _sorted_idx = sortperm(θ_k);
        θ_k[_sorted_idx][findall(cumsum(w[_sorted_idx]) .< α)[end]])

    function _return_posteriors(; l::Int)
        particles_l = NamedArray(
            [param_constrain(ℓπ_vec[:l => l].model, particles[:l => l][:n => n];
                    include_tp=true,
                    include_gq=true, rng=StanRNG(ℓπ_vec[:l => l].model, SEED)
                    ) for n in 1:N] |> vecvec2mat,
            (1:N,
                param_names(ℓπ_vec[:l => l].model;
                    include_tp=true, include_gq=true)),
            (:n, :d))
        L_vec, M_vec, U_vec = Float64[], Float64[], Float64[]
        for s in 0:data_0["S"]
            if s == 0
                θ_s = Vector{Float64}([particles_l[:n => n, :d => "predicted_score.$(data_0["T"]).$(s)"]
                        for n=1:N, s=1:data_0["S"]] * data_0["state_weights"])
            else
                θ_s = particles_l[:d => "predicted_score.$(data_0["T"]).$(s)"]
            end
            _L = _weighted_quantile(θ_s, weights[:l => l]; α=0.1)
            _M = θ_s' * weights[:l => l]
            _U = _weighted_quantile(θ_s, weights[:l => l]; α=1-0.1)
            push!(L_vec, _L); push!(M_vec, _M); push!(U_vec, _U)
        end
        (L_vec, M_vec, U_vec)
    end
    
    (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results_2_pos
    plot_mat = NamedArray(zeros(1+data_0["S"], 1+2L, 3),
        (0:data_0["S"], -L:L, [:L, :M, :U]), (:s, :l, :type))
    @showprogress for l in 0:L#LinRange(0, L, L) .|> round .|> Int
        (L_vec, M_vec, U_vec) = _return_posteriors(l=l)
        plot_mat[:l => l, :type => :L] = L_vec
        plot_mat[:l => l, :type => :M] = M_vec
        plot_mat[:l => l, :type => :U] = U_vec
    end

    (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results_2_neg
    @showprogress for l in 1:L#LinRange(0, L, L)[2:end] .|> round .|> Int
        (L_vec, M_vec, U_vec) = _return_posteriors(l=l)
        plot_mat[:l => -l, :type => :L] = L_vec
        plot_mat[:l => -l, :type => :M] = M_vec
        plot_mat[:l => -l, :type => :U] = U_vec
    end

    global plot_mat_2 = plot_mat
end

In [ ]:
let
    plot_mat = plot_mat_2
    h = 0.42
    #sorted_idx_0 = [abbrev2int[s] for s in ["UT", "AL", "TX", "AK", "GA", "CO", "MD"]]
    sorted_idx_0 = sortperm(plot_mat[:l => 0, :type => :M]) .- 1
    
    hline([0.5], color=:black, alpha=0.2, label="")
    for (i, s_i) in sorted_idx_0 |> enumerate
        plot!(LinRange(i-h, i+h, size(plot_mat, 2)),
            plot_mat[:s => s_i, :type => :L],
            label=i == 1 ? "80% CI, lined-up horizontally (" * L"\leftarrow" * "tighter; looser" * L"\rightarrow" * ")" : "",
            fillrange=plot_mat[:s => s_i, :type => :U],
            color=s_i == 0 ? :black : :gray, alpha=0.5, linewidth=0)
        scatter!([i], [plot_mat[:s => s_i, :l => 0, :type => :M]],
            label=i == 1 ? "Original final score" : "",
            color=:black, ms=3, mswidth=1)

        options = (; color=:black, markershape=:hline, ms=3, mswidth=1, label=i == 1 ? "Prior forecast" : "")
        # R code: national_mu_prior <- weighted.mean(arm::invlogit(mu_b_prior), state_weights)
        m_s = s_i == 0 ? logistic.([results_2_pos[:ℓπ_vec][:l => 0].data["mu_b_prior"][s_for] for s_for in 1:data_0["S"]])' * data_0["state_weights"] : logistic(results_2_pos[:ℓπ_vec][:l => 0].data["mu_b_prior"][s_i])
        scatter!([i], [m_s]; options...)
    end
    S_vec = [s_i == 0 ? "National" : int2full[s_i] for s_i in sorted_idx_0]
    global plt_4 = plot!(size=(670,400), frame=:semi,
        xrot=-90, xticks=(1:length(S_vec), S_vec),
        bottommargin=10Plots.mm,
        #ylim=(0,1),
        legend=false,
        title="Adjusting " * L"\texttt{random~walk~scale}"
    )
end

In [ ]:
savefig(joinpath("img", "random-walk-scale.pdf"));

In [ ]:
# perturb_i = 4

Random.seed!(1); let
    
    function _plot(s::String)

        df_s = filter(row -> row.state == s, df)
        
        (; ℓπ_vec, names_vec, D_vec,
            particles, weights,
            ESS, k̂, mcmc_flag,
            R, N, L,
            times) = results_4
        N_cut = N
        
        plts = []; @showprogress for l in [1, 10, L]
            
            c_l = :blue
            
            plot_data(s)
            
            function _plot_trend(θ::Matrix{Float64}, w::Vector{Float64})
                _weighted_quantile(θ_k::Vector{Float64}, w::Vector{Float64}; α::Float64) = (
                    sorted_idx = sortperm(θ_k);
                    θ_k[sorted_idx][findall(cumsum(w[sorted_idx]) .< α)[end]])
            
                plot!(START:END, [_weighted_quantile(θ[:,k], w; α=0.1/2) for k in axes(θ,2)],
                    fillrange=[_weighted_quantile(θ[:,k], w; α=1-0.1/2) for k in axes(θ,2)],
                    color=c_l, alpha=0.15, linewidth=0)
                plot!(START:END, _sum(θ .* w; dims=1), color=c_l, alpha=1.)
            end
            
            particles_l = NamedArray(
                [param_constrain(ℓπ_vec[:l => l].model, particles[:l => l][:n => n];
                        include_tp=true,
                        include_gq=true, rng=StanRNG(ℓπ_vec[:l => l].model, SEED)
                        ) for n in 1:N_cut] |> vecvec2mat,
                (1:N_cut,
                    param_names(ℓπ_vec[:l => l].model;
                        include_tp=true, include_gq=true)),
                (:n, :d))
            
            θ = zeros(N_cut, data_0["T"])
            for (n,t) in Iterators.product(1:N_cut, 1:data_0["T"])
                θ[n,t] = particles_l[:n => n, :d => "predicted_score.$(t).$(abbrev2int[s])"]
            end
            _plot_trend(θ, weights[:l => l])

            n_i = ℓπ_vec[:l => l].data["n_democrat_state"][end]
            d_i = ℓπ_vec[:l => l].data["n_two_share_state"][end]
            if s == "PA"
                a_i = (.9 + (1 - .9)*(n_i/maximum(df_s.n_respondents)))
                x_i = (START:END)[ℓπ_vec[:l => l].data["day_state"][end]]
                y_i = n_i / d_i
                scatter!([x_i], [y_i], mswidth=1, color=:red, ms=4a_i, alpha=a_i)
                annotate!(x_i, y_i, text("\n $(n_i)/$(d_i)", 7, :left))
            end
            
            push!(plts, plot!(title="Step $(l)/$(L)"))
            # l < L && push!(plts, (plot(grid=false, axis=false); annotate!(0, .6, L"\ldots" * "\n" * L"\rightarrow")))
        end
        plts
    end
    
    #plot(_plot("CA"),
    #    (_plot("AR"); plot!(xlabel="")),
    #    (_plot("OH"); plot!(xlabel="")),
    #    layout=grid(1,3))
    #for s in keys(abbrev2int)
    #    @info s; _plot(s) |> display; a
    #end
    plts = _plot("PA")
    plot(plts..., layout=grid(1, length(plts)))
end;

In [ ]:
plt_5 = plot!(size=(700, 270), rightmargin=2Plots.mm, bottommargin=6Plots.mm)

In [ ]:
savefig(joinpath("img", "data-insersion.pdf"));